In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#!conda install -c districtdatalabs yellowbrick
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library



print('Libraries imported.')

C:\Users\schne\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


Libraries imported.


In [2]:

import requests
import geopy

from geopy.geocoders import ArcGIS
from functools import partial
from geopy.extra.rate_limiter import RateLimiter
#from tqdm import tqdm
#tqdm.pandas()


In [3]:
#Download the wiki table to a readable df format
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
print(df.shape)
df.head()


(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"



### Data Preprocessing

In [4]:
#Remove entries where Borough = "Not Assigned"
df = df[df.Borough != "Not assigned"]

In [5]:
#Check if any neighborhoods are still unassigned 
df=df.rename(columns={"Neighbourhood" : "Neighborhood"})
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
#If neighborhoods still contain "Not Assigned", but have an associated Borough, run the code below to transform Neighborhood NA's to Burough name
df['Neighborhood'].replace({'Not assigned':df['Borough']}, inplace = True)

In [7]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
#Reset df index
df = df.reset_index(drop=True)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Check the shape of the data frame

In [9]:
df.shape

(103, 3)

In [10]:
### Convert Neighborhood strings to lists
lst_col='Neighborhood'

df = df.assign(**{lst_col:df[lst_col].str.split(', ')})
# Check df 
df

### Explode Neighborhood lists to unique columns
df=df.explode('Neighborhood')


print(df.shape)
df
df = df.reset_index(drop=True)
df.head(10)

(217, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor
5,M6A,North York,Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park
7,M7A,Downtown Toronto,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M9A,Etobicoke,Humber Valley Village


In [11]:
#Rename df column (Wiki data has changed since V1)
df=df.rename(columns={'Post Code' : 'Postal Code'})

In [12]:
df_latlng=pd.read_csv("http://cocl.us/Geospatial_data")

In [13]:
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df=df.merge(df_latlng, left_on = "Postal Code", right_on = "Postal Code")

In [39]:
df['Neighborhood'].unique().shape

(208,)

In [16]:
data_t=pd.read_csv(r'C:\Users\schne\Desktop\IBM Data Science Certification\Python v2\Applied Data Science Capstone\Notebooks\Data\neighbourhood-profiles-2016-csv.csv')
print("Categories : "+data_t.Category.unique())


print("Topics : " + data_t.Topic.unique())


print("Characteristics : " + data_t.Characteristic.unique())

['Categories : Neighbourhood Information' 'Categories : Population'
 'Categories : Families, households and marital status'
 'Categories : Language' 'Categories : Income'
 'Categories : Immigration and citizenship'
 'Categories : Visible minority' 'Categories : Ethnic origin'
 'Categories : Aboriginal peoples' 'Categories : Education'
 'Categories : Housing' 'Categories : Language of work'
 'Categories : Labour' 'Categories : Journey to work'
 'Categories : Mobility']
['Topics : Neighbourhood Information' 'Topics : Population and dwellings'
 'Topics : Age characteristics'
 'Topics : Household and dwelling characteristics'
 'Topics : Marital status' 'Topics : Family characteristics'
 'Topics : Household type' 'Topics : Family characteristics of adults'
 'Topics : Knowledge of official languages'
 'Topics : First official language spoken' 'Topics : Mother tongue'
 'Topics : Knowledge of languages' 'Topics : Income of households in 2015'
 'Topics : Language spoken most often at home'
 'To

In [17]:
data_t=data_t.drop(columns={"_id","Data Source","Category"})
data_t2=data_t.transpose().reset_index()
data_t.head()

,Topic,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designation,NIA,No Designation,No Designation,Emerging Neighbourhood,No Designation,No Designation,No Designation

In [18]:
data_t2=data_t.transpose().reset_index()


In [19]:
data_t3=data_t2.loc[3:].rename(columns=data_t2.loc[1]).reset_index(drop=True)
data_t3=data_t3.rename(columns={"Characteristic":"Neighborhood"})
data_t3.head()

Neighborhood Neighbourhood Number TSNS2020 Designation  \
0               Agincourt North                  129       No Designation   
1  Agincourt South-Malvern West                  128       No Designation   
2                     Alderwood                   20       No Designation   
3                         Annex                   95       No Designation   
4             Banbury-Don Mills                   42       No Designation   

  Population, 2016 Population, 2011 Population Change 2011-2016  \
0           29,113           30,279                      -3.90%   
1           23,757           21,988                       8.00%   
2           12,054           11,904                       1.30%   
3           30,526           29,177                       4.60%   
4           27,695           26,918                       2.90%   

  Total private dwellings Private dwellings occupied by usual residents  \
0                   9,371                                         9,120   
1                   8,535                                         8,136   
2                   4,732                                         4,616   
3                  18,109                                        15,934   
4                  12,473                                        12,124   

  Population density per square kilometre Land area in square kilometres  \
0                                   3,929                           7.41   
1                                   3,034                           7.83   
2                                   2,435                           4.95   
3                                  10,863                           2.81   
4                                   2,775                           9.98   

  Children (0-14 years) Youth (15-24 years) Working Age (25-54 years)  \
0                 3,840               3,705                    11,305   
1                 3,075               3,360                     9,965   
2                 1,760               1,235                     5,220   
3                 2,360               3,750                    15,040   
4                 3,605               2,730                    10,810   

  Pre-retirement (55-64 years) Seniors (65+ years) Older Seniors (85+ years)  \
0                        4,230               6,045                       925   
1                        3,265               4,105                       555   
2                        1,825               2,015                       320   
3                        3,480               5,910                     1,040   
4                        3,555               6,975                     1,640   

  Male: 0 to 04 years Male: 05 to 09 years Male: 10 to 14 years  \
0                 660                  695                  660   
1                 575                  540                  460   
2                 360                  270                  225   
3                 445                  365                  325   
4                 570                  660                  675   

  Male: 15 to 19 years Male: 20 to 24 years Male: 25 to 29 years  \
0                  840                 1015                 1015   
1                  780                 1000                 1045   
2                  285                  355                  355   
3                  465                 1215                 2080   
4                  715                  700                  645   

  Male: 30 to 34 years Male: 35 to 39 years Male: 40 to 44 years  \
0                  835                  680                  760   
1                  820                  625                  610   
2                  410                  455                  420   
3                 1610                 1055                  835   
4                  735                  735                  815   

  Male: 45 to 49 years Male: 50 to 54 years Male: 55 to 59 years  \
0                  890              

In [20]:
data_t4=data_t3.iloc[0: ,0:58]
print(data_t4.shape)
data_t4.sort_values('Neighborhood').head()


(140, 58)


,Neighborhood,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),Youth (15-24 years),Working Age (25-54 years),Pre-retirement (55-64 years),Seniors (65+ years),Older Seniors (85+ years),Male: 0 to 04 years,Male: 05 to 09 years,Male: 10 to 14 years,Male: 15 to 19 years,Male: 20 to 24 years,Male: 25 to 29 years,Male: 30 to 34 years,Male: 35 to 39 years,Male: 40 to 44 years,Male: 45 to 49 years,Male: 50 to 54 years,Male: 55 to 59 years,Male: 60 to 64 years,Male: 65 to 69 years,Male: 70 to 74 years,Male: 75 to 79 years,Female: 10 to 14 years,Male: 80 to 84 years,Male: 85 to 89 years,Male: 90 to 94 years,Male: 95 to 99 years,Male: 100 years and over,Female: 0 to 04 years,Female: 05 to 09 years,Female: 15 to 19 years,Female: 20 to 24 years,Female: 25 to 29 years,Female: 30 to 34 years,Female: 35 to 39 years,Female: 40 to 44 years,Female: 45 to 49 years,Female: 50 to 54 years,Female: 55 to 59 years,Female: 60 to 64 years,Female: 65 to 69 years,Female: 70 to 74 years,Female: 75 to 79 years,Female: 80 to 84 years,Female: 85 to 89 years,Female: 90 to 94 years,Female: 95 to 99 years,Female: 100 years and over
0,Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840","3,705","11,305","4,230","6,045",925,660,695,660,840,1015,1015,835,680,760,890,1160,1060,925,925,590,490,595,380,210,100,25,0,620,620,865,975,1005,935,775,910,1060,1290,1165,1070,985,690,575,485,350,160,60,10
1,Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075","3,360","9,965","3,265","4,105",555,575,540,460,780,1000,1045,820,625,610,760,970,850,710,630,425,350,475,240,155,45,20,0,530,495,690,895,975,835,715,745,885,975,915,795,690,450,405,350,205,100,20,0
2,Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760","1,235","5,220","1,825","2,015",320,360,270,225,285,355,355,410,455,420,440,515,540,390,300,205,155,255,105,65,35,10,0,320,325,290,310,350,430,450,445,430,515,485,400,325,210,180,210,130,70,5,5
3,Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360","3,750","15,040","3,480","5,910","1,040",445,365,325,465,1215,2080,1610,1055,835,850,920,855,755,780,640,485,355,335,230,80,25,5,480,400,550,1520,2265,1675,1040,915,855,935,915,940,950,700,565,425,345,260,90,25
4,Banbury-Don Mills,42,No Designation,"27,695","26,918",2.90%,"12,473","12,124","2,775",9.98,"3,605","2,730","10,810","3,555","6,975","1,640",570,660,675,715,700,645,735,735,815,1010,1110,930,730,715,570,505,600,435,340,175,20,5,500,600,660,650,745,860,895,975,1170,1115,1005,895,955,790,730,650,615,360,105,20


### Convert Neighborhood strings to lists
lst_col='Neighborhood'

data_t4 = data_t4.assign(**{lst_col:data_t4[lst_col].str.split('-')})

### Explode Neighborhood lists to unique columns

data_t4=data_t4.explode('Neighborhood')


print(data_t4.shape)

data_t4 = data_t4.reset_index(drop=True)
data_t4.head()

In [21]:
with open (r'C:\Users\schne\Desktop\IBM Data Science Certification\Python v2\Applied Data Science Capstone\Neighbourhoods.json') as f:
    data = json.load(f)

data2 = data['features']

In [22]:
data3 = json_normalize(data2)

cols = data3.columns.tolist()
cols = cols[7:8]+cols[11:13]  + cols[1:2] + cols[14:16]
data3=data3[cols]
print(cols)
data3=data3.rename(columns = {"properties.AREA_NAME":"Neighborhood","properties.LATITUDE":"Latitude","properties.LONGITUDE":"Longitude"})
print(data3.shape)
data3.head()

<ipython-input-22-ce1feb4ea953>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data3 = json_normalize(data2)


['properties.AREA_NAME', 'properties.LONGITUDE', 'properties.LATITUDE', 'properties._id', 'properties.Shape__Area', 'properties.Shape__Length']
(140, 6)


,Neighborhood,Longitude,Latitude,properties._id,properties.Shape__Area,properties.Shape__Length
0,Wychwood (94),-79.425515,43.676919,6441,3.217960e+06,7515.779658
1,Yonge-Eglinton (100),-79.403590,43.704689,6442,3.160334e+06,7872.021074
2,Yonge-St.Clair (97),-79.397871,43.687859,6443,2.222464e+06,8130.411276
3,York University Heights (27),-79.488883,43.765736,6444,2.541821e+07,25632.335242
4,Yorkdale-Glen Park (31),-79.457108,43.714672,6445,1.156669e+07,13953.408098


In [23]:
data32 = data3['Neighborhood'].str.rsplit(" (",expand=True)
data32=data32.rename(columns={0:'Neighborhood'})
data32=data32.drop(columns=[1,2])
print(data32.shape)
data32.head(10)


(140, 1)


,Neighborhood
0,Wychwood
1,Yonge-Eglinton
2,Yonge-St.Clair
3,York University Heights
4,Yorkdale-Glen Park
5,Lambton Baby Point
6,Lansing-Westgate
7,Lawrence Park North
8,Lawrence Park South
9,Leaside-Bennington


In [24]:
data33=data3.merge(data32, left_index=True,right_index=True) 
print(data33.shape)
data33.head()

(140, 7)


,Neighborhood_x,Longitude,Latitude,properties._id,properties.Shape__Area,properties.Shape__Length,Neighborhood_y
0,Wychwood (94),-79.425515,43.676919,6441,3.217960e+06,7515.779658,Wychwood
1,Yonge-Eglinton (100),-79.403590,43.704689,6442,3.160334e+06,7872.021074,Yonge-Eglinton
2,Yonge-St.Clair (97),-79.397871,43.687859,6443,2.222464e+06,8130.411276,Yonge-St.Clair
3,York University Heights (27),-79.488883,43.765736,6444,2.541821e+07,25632.335242,York University Heights
4,Yorkdale-Glen Park (31),-79.457108,43.714672,6445,1.156669e+07,13953.408098,Yorkdale-Glen Park


In [25]:

cols = data33.columns.tolist()
cols =cols[6:7] + cols[1:3] + cols[3:6]
data33=data33[cols]
data33=data33.rename(columns = {"Neighborhood_y":"Neighborhood"})
data33.head()

,Neighborhood,Longitude,Latitude,properties._id,properties.Shape__Area,properties.Shape__Length
0,Wychwood,-79.425515,43.676919,6441,3.217960e+06,7515.779658
1,Yonge-Eglinton,-79.403590,43.704689,6442,3.160334e+06,7872.021074
2,Yonge-St.Clair,-79.397871,43.687859,6443,2.222464e+06,8130.411276
3,York University Heights,-79.488883,43.765736,6444,2.541821e+07,25632.335242
4,Yorkdale-Glen Park,-79.457108,43.714672,6445,1.156669e+07,13953.408098


### Convert Neighborhood strings to lists
lst_col='Neighborhood'

data33 = data33.assign(**{lst_col:data33[lst_col].str.split('-')})

### Explode Neighborhood lists to unique columns

data33=data33.explode('Neighborhood')


print(data33.shape)

data33 = data33.reset_index(drop=True)
data33.head()

In [26]:
data33.sort_values('Neighborhood')

,Neighborhood,Longitude,Latitude,properties._id,properties.Shape__Area,properties.Shape__Length
74,Agincourt North,-79.266712,43.805441,6515,1.395145e+07,17159.740667
75,Agincourt South-Malvern West,-79.265612,43.788658,6516,1.511736e+07,21320.849547
76,Alderwood,-79.541611,43.604937,6517,9.502180e+06,12667.013917
77,Annex,-79.404001,43.671585,6518,5.337192e+06,10513.883143
78,Banbury-Don Mills,-79.349718,43.737657,6519,1.924897e+07,25141.572290
79,Bathurst Manor,-79.456055,43.764813,6520,9.133974e+06,20183.925841
80,Bay Street Corridor,-79.385721,43.657511,6521,3.459075e+06,9594.336045
81,Bayview Village,-79.377117,43.776361,6522,9.902533e+06,13169.020187
82,Bayview Woods-Steeles,-79.382118,43.796802,6523,7.853671e+06,11439.166125
83,Bedford Park-Nortown,-79.420227,43.731486,6524,1.057626e+07,17755.857560


In [27]:
##data33.Neighborhood = data33.Neighborhood.str.encode('utf-8')
print(data33.dtypes)
#df.Neighborhood = df.Neighborhood.str.encode('utf-8')
print(data_t4.dtypes)
print(data33.sort_values('Neighborhood').head())
print(data_t4.sort_values('Neighborhood').head())

Neighborhood                 object
Longitude                   float64
Latitude                    float64
properties._id                int64
properties.Shape__Area      float64
properties.Shape__Length    float64
dtype: object
Neighborhood                                     object
Neighbourhood Number                             object
TSNS2020 Designation                             object
Population, 2016                                 object
Population, 2011                                 object
Population Change 2011-2016                      object
Total private dwellings                          object
Private dwellings occupied by usual residents    object
Population density per square kilometre          object
Land area in square kilometres                   object
Children (0-14 years)                            object
Youth (15-24 years)                              object
Working Age (25-54 years)                        object
Pre-retirement (55-64 years)              

In [47]:
toronto_data=pd.merge(data33.sort_values("Neighborhood"),data_t4.sort_values("Neighborhood"),left_on="Neighborhood",right_on='Neighborhood')

In [52]:
toronto_data.head()

,Neighborhood,Longitude,Latitude,properties._id,properties.Shape__Area,properties.Shape__Length,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),Youth (15-24 years),Working Age (25-54 years),Pre-retirement (55-64 years),Seniors (65+ years),Older Seniors (85+ years),Male: 0 to 04 years,Male: 05 to 09 years,Male: 10 to 14 years,Male: 15 to 19 years,Male: 20 to 24 years,Male: 25 to 29 years,Male: 30 to 34 years,Male: 35 to 39 years,Male: 40 to 44 years,Male: 45 to 49 years,Male: 50 to 54 years,Male: 55 to 59 years,Male: 60 to 64 years,Male: 65 to 69 years,Male: 70 to 74 years,Male: 75 to 79 years,Female: 10 to 14 years,Male: 80 to 84 years,Male: 85 to 89 years,Male: 90 to 94 years,Male: 95 to 99 years,Male: 100 years and over,Female: 0 to 04 years,Female: 05 to 09 years,Female: 15 to 19 years,Female: 20 to 24 years,Female: 25 to 29 years,Female: 30 to 34 years,Female: 35 to 39 years,Female: 40 to 44 years,Female: 45 to 49 years,Female: 50 to 54 years,Female: 55 to 59 years,Female: 60 to 64 years,Female: 65 to 69 years,Female: 70 to 74 years,Female: 75 to 79 years,Female: 80 to 84 years,Female: 85 to 89 years,Female: 90 to 94 years,Female: 95 to 99 years,Female: 100 years and over
0,Agincourt North,-79.266712,43.805441,6515,1.395145e+07,17159.740667,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840","3,705","11,305","4,230","6,045",925,660,695,660,840,1015,1015,835,680,760,890,1160,1060,925,925,590,490,595,380,210,100,25,0,620,620,865,975,1005,935,775,910,1060,1290,1165,1070,985,690,575,485,350,160,60,10
1,Agincourt South-Malvern West,-79.265612,43.788658,6516,1.511736e+07,21320.849547,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075","3,360","9,965","3,265","4,105",555,575,540,460,780,1000,1045,820,625,610,760,970,850,710,630,425,350,475,240,155,45,20,0,530,495,690,895,975,835,715,745,885,975,915,795,690,450,405,350,205,100,20,0
2,Alderwood,-79.541611,43.604937,6517,9.502180e+06,12667.013917,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760","1,235","5,220","1,825","2,015",320,360,270,225,285,355,355,410,455,420,440,515,540,390,300,205,155,255,105,65,35,10,0,320,325,290,310,350,430,450,445,430,515,485,400,325,210,180,210,130,70,5,5
3,Annex,-79.404001,43.671585,6518,5.337192e+06,10513.883143,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360","3,750","15,040","3,480","5,910","1,040",445,365,325,465,1215,2080,1610,1055,835,850,920,855,755,780,640,485,355,335,230,80,25,5,480,400,550,1520,2265,1675,1040,915,855,935,915,940,950,700,565,425,345,260,90,25
4,Banbury-Don Mills,-79.349718,43.737657,6519,1.924897e+07,25141.572290,42,No Designation,"27,695","26,918",2.90%,"12,473","12,124","2,775",9.98,"3,605","2,730","10,810","3,555","6,975","1,640",570,660,675,715,700,645,735,735,815,1010,1110,930,730,715,570,505,600,435,340,175,20,5,500,600,660,650,745,860,895,975,1170,1115,1005,895,955,790,730,650,615,360,105,20


In [ ]:

import requests
import geopy

from geopy.geocoders import ArcGIS
from functools import partial
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()



In [ ]:
geolocator = geopy.geocoders.ArcGIS(username="WWSscript",password="Z3MWMrnEhXid4Z3",referer="https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer",token_lifetime=5)
geolocator

In [ ]:
# initialize your variable to None
lat_lng_coords = None

In [ ]:
from functools import partial
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
df['location'] = df['{},{}'.format(df['Borough'], df['Neighborhood'])].apply(geocode,axis=0)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
